# Последовательный анализ

In [ ]:
!python3 -m pip install sprt

## Библиотеки

In [ ]:
import numpy as np
import sprt
import scipy.stats as st
from statsmodels.stats.proportion import proportions_ztest

import matplotlib.pyplot as plt

## Warmup

### Генерация данных

- $p_L$ - нижняя граница
- $p_U$ - верхняя граница
- $\alpha$ - уровень значимости, допускаемая вероятность ошибки первого рода: отвергнуть верную $H_0$ (при $p \le p_L$)
- $\beta$ - допускаемая вероятность ошибки второго рода: принять неверную $H_0$ (при $p \ge p_U$)

In [ ]:
pl = 0.45
pu = 0.55
alpha = beta = 0.05
A = (1 - beta) / alpha
B = beta / (1 - alpha)

def a_m(m):
    return (np.log(B) + m * np.log((1 - pl) / (1 - pu))) / (np.log(pu / pl) - np.log((1 - pu) / (1 - pl)))

def r_m(m):
    return (np.log(A) + m * np.log((1 - pl) / (1 - pu))) / (np.log(pu / pl) - np.log((1 - pu) / (1 -pl)))

In [ ]:
plt.plot(range(100), [a_m(i) for i in range(100)], label='accept', color='red')
plt.plot(range(100), [r_m(i) for i in range(100)], label='reject', color='green')
plt.xlabel('m')
plt.ylabel('$d_m$')
plt.legend(loc='best')

plt.title('Regection and acceptance lines')

plt.show()

### Z-критерий меток для доли (лекция 2: параметрические гипотезы) и последовательный анализ для проверки $p$

Сравним результаты последовательного анализа и z-теста меток для доли

Последовательный анализ:

$H_0:\ p \le p_L$ 

$H_1:\ p \ge p_U$

Критерий меток доли (левосторонняя альтернатива):

$H_0:\ p = p_0$ 

$H_1:\ p < p_0$

#### Генерация данных ($Ber(p)$) и проверка z-критерием меток

In [ ]:
def generate_and_test(real_p, test_func, maxm=500, seed=42):
    """
    Генерация выборки из бернуллиевских величин, генерация останавливается, когда при последовательном анализе мы отвергаем
    или принимаем H_0 и одновременно то же делает z-test
    :param real_p - p
    :param test_func - функция для проверки z-теста, возвращает p-value
    :param maxm - максимальный размер выборки
    :return полученная выборка, массив p-value, номер первой итерации, где график вышел за пределы полосы
    """
    rs = np.random.RandomState(seed)
    sample = []
    pvals = []
    m = 0
    test_iter_num = -1 # номер итерации, где отвергается гипотеза по z-тесту
    iter_num = -1 # номер итерации, где мы выходим за пределы области безразличия (между двумя accept и reject)
    while m < maxm and (iter_num < 0 or test_iter_num < 0):
        m += 1
        x = rs.uniform() <= real_p
        sample += [x]
        pvals.append(test_func(sample))
        if iter_num < 0 and (np.sum(sample) <= a_m(m) or np.sum(sample) >= r_m(m)):
            iter_num = m
            
        if pvals[-1] < 0.05:
            test_iter_num = m

    return sample, pvals, iter_num

In [ ]:
def plot_seq_ztest_results(sample, bin_test):
    max_l = len(sample)
    fig = plt.figure(figsize=(15, 5))
    ax1 = fig.add_subplot(121)
    ax1.plot(np.cumsum(sample), label='actual d_m')
    ax1.plot(range(max_l), [a_m(i) for i in range(max_l)], label='accept', color='green')
    ax1.plot(range(max_l), [r_m(i) for i in range(max_l)], label='reject', color='red')
    ax1.legend(loc='best')
    ax1.set_xlabel('m')
    ax1.set_ylabel('$d_m$')
    ax1.set_title('Sequential analysis')

    ax2 = fig.add_subplot(122)
    ax2.plot([0, 250], [0.05, 0.05])
    ax2.plot(bin_test)
    ax2.set_xlabel('m')
    ax2.set_ylabel('p-value')
    ax2.set_title('Z-tests')

#### $p=0.4$

In [ ]:
sample, bin_test, _ = generate_and_test(0.4, test_func=lambda x: proportions_ztest(np.sum(x), len(x), 0.5, 'smaller')[1])
plot_seq_ztest_results(sample, bin_test)

#### $p=0.6$

In [ ]:
sample, bin_test, _ = generate_and_test(0.6, test_func=lambda x: proportions_ztest(np.sum(x), len(x), 0.5, 'smaller')[1])
plot_seq_ztest_results(sample, bin_test)

#### $p=0.5$

In [ ]:
sample, bin_test, _ = generate_and_test(0.5, test_func=lambda x: proportions_ztest(np.sum(x), len(x), 0.5, 'smaller')[1], maxm=1000)
plot_seq_ztest_results(sample, bin_test)

#### Рассмотрим разный random_state

In [ ]:
max_l = 0
bin_tests = []
iter_nums = []
for i in range(30):
    sample_, bin_test, iter_num = generate_and_test(0.4, seed=i, test_func=lambda x: proportions_ztest(np.sum(x), len(x), 0.5, 'smaller' )[1])
    max_l_ = len(sample_)
    max_l = max(max_l_, max_l)
    iter_nums.append(iter_num)
    bin_tests.append(bin_test)
print(max_l)

plt.figure(figsize=(15, 5))
for t in bin_tests:
    plt.plot(t)        
plt.xlabel('m')
plt.ylabel('p-value', fontdict={'size': 15})

#### Разница в размерах итоговых выборок

In [ ]:
min_num = []
for p in bin_tests:
    min_num.append(np.where(np.array(p) <= 0.05)[0][0])
min_num = np.array(min_num)
plt.hist(np.array(iter_nums), bins=30)

plt.show()

#### Последовательный тест Вальда из коробки
- $h_0 == H_0$ 
- $h_1 == H_1$

In [ ]:
rs = np.random.RandomState(42)
values = rs.uniform(size=500) <= 0.5
test = sprt.SPRTBinomial(alpha=alpha, beta=beta, h0=0.45, h1=0.55, values=values)
test.plot()

plt.show()

## Метилфенидат и синдром дефицита внимания

24 умственно отсталых ребёнка с СДВГ в течение недели принимали либо метилфенидат, либо плацебо, а в конце недели проходили тест на способность к подавлению импульсивных поведенческих реакций. На втором этапе плацебо и препарат менялись, после недельного курса каждый испытуемые проходили второй тест.

Каков эффект препарата?

### Данные

In [ ]:
att = np.loadtxt('https://raw.githubusercontent.com/Intelligent-Systems-Phystech/psad/master/seminars/sem10/data/ADHD.txt', delimiter=' ', skiprows=1)


plt.scatter(att[:, 0], att[:, 1])
plt.xlabel('Placebo')
plt.ylabel('Methylphenidate')
_ = plt.plot([min(att[:,0]), max(att[:,0])], [min(att[:, 1]), max(att[:, 1])])

### Нормальность дельт

In [ ]:
deltas = att[:, 1] - att[:, 0]
plt.hist(deltas)
_ = plt.title('Deltas histogram')

In [ ]:
st.shapiro(deltas)

### Последовательный анализ Вальда для нормального распределенияи

Рассмотрим, как меняется коридор безразличия при изменении $\alpha$ и $\beta$ (предполагаем, что дисперсия известна, так как иначе это будет t-тест, а не z-тест).

$H_0: \mu \le \mu_L$

$H_1: \mu \ge \mu_U$

- $p_L$ - нижняя граница
- $p_U$ - верхняя граница
- $\alpha$ - уровень значимости, допускаемая вероятность ошибки первого рода: отвергнуть верную $H_0$ (при $p \le p_L$)
- $\beta$ - допускаемая вероятность ошибки второго рода: принять неверную $H_0$ (при $p \ge p_U$)

In [ ]:
mean = np.mean(deltas)
var = np.std(deltas, ddof=1)**2

In [ ]:
test = sprt.SPRTNormal(alpha=0.05, beta=0.05, h0=mean / 3,  h1=2 * mean / 3, values=deltas, variance=var)
test.plot()

In [ ]:
test = sprt.SPRTNormal(alpha=0.15, beta=0.05, h0=mean / 3,  h1=2 * mean / 3, values=deltas, variance=var)
test.plot()

In [ ]:
test = sprt.SPRTNormal(alpha=0.05, beta=0.15, h0=mean / 3,  h1=2 * mean / 3, values=deltas, variance=var)
test.plot()

In [ ]:
test = sprt.SPRTNormal(alpha=0.05, beta=0.05, h0=mean-mean / 1.5,  h1=mean+mean/1.5, values=deltas, variance=var)
test.plot()